In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install --upgrade pip
!pip install scikit-learn==1.1.2 scipy==1.9.1 missingpy==0.2.0


In [6]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=8aef08fd4b036ed0c688c5d97a642844543a433915a44eef3878cf3388332390
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [7]:

import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [8]:
import numpy as np
import matplotlib as plt
import seaborn as sns

In [9]:
file_path = "/content/drive/MyDrive/Big Data /Kaggle/data/train.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True)

df.show()

+------------+-----------+---------------+------+--------+------+-----------------+------------------------+-------------------+----------------+----------+--------------------+------------------+--------------+---------------+-----------------+----------------+----------------+------+------+------+----------------+-------------+---------+---------+-----------+-----------+--------------+-----------+-------------+---------+---------+-----------+------------+-------------+--------+--------+------------+----------+-------+-----------+-------+-------------------+-------------------+------------------------+-----------------------+----------------------+------+-----------+----------+
|encounter_id|patient_nbr|           race|gender|     age|weight|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|payer_code|   medical_specialty|num_lab_procedures|num_procedures|num_medications|number_outpatient|number_emergency|number_inpatient|diag_1|diag_2|diag_3|number_diagn

In [10]:
df.select("weight").show()

+------+
|weight|
+------+
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
|     ?|
+------+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import col, when

def count_question_marks_per_column(df):

    counts = {}

    for column in df.columns:
        count_question_mark = df.filter(col(column) == "?").count()
        counts[column] = count_question_mark

    return counts
question_mark_counts = count_question_marks_per_column(df)

# Hiển thị kết quả
for column, count in question_mark_counts.items():
    if count >0:
      print(f"Số lượng giá trị '?' trong cột '{column}': {count}")


Số lượng giá trị '?' trong cột 'race': 1878
Số lượng giá trị '?' trong cột 'weight': 68671
Số lượng giá trị '?' trong cột 'payer_code': 30085
Số lượng giá trị '?' trong cột 'medical_specialty': 34525
Số lượng giá trị '?' trong cột 'diag_1': 17
Số lượng giá trị '?' trong cột 'diag_2': 290
Số lượng giá trị '?' trong cột 'diag_3': 1146


In [12]:
# chuyển giá trị "?" về dạng null
df = df.withColumn("weight", when(col("weight") == "?", None).otherwise(col("weight")))
df = df.withColumn("race", when(col("race") == "?", None).otherwise(col("race")))
df = df.withColumn("payer_code", when(col("payer_code") == "?", None).otherwise(col("payer_code")))
df = df.withColumn("medical_specialty", when(col("medical_specialty") == "?", None).otherwise(col("medical_specialty")))
df = df.withColumn("diag_1", when(col("diag_1") == "?", None).otherwise(col("diag_1")))
df = df.withColumn("diag_2", when(col("diag_2") == "?", None).otherwise(col("diag_2")))
df = df.withColumn("diag_3", when(col("diag_3") == "?", None).otherwise(col("diag_3")))

df.show(10)

+------------+-----------+---------------+------+--------+------+-----------------+------------------------+-------------------+----------------+----------+--------------------+------------------+--------------+---------------+-----------------+----------------+----------------+------+------+------+----------------+-------------+---------+---------+-----------+-----------+--------------+-----------+-------------+---------+---------+-----------+------------+-------------+--------+--------+------------+----------+-------+-----------+-------+-------------------+-------------------+------------------------+-----------------------+----------------------+------+-----------+----------+
|encounter_id|patient_nbr|           race|gender|     age|weight|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|payer_code|   medical_specialty|num_lab_procedures|num_procedures|num_medications|number_outpatient|number_emergency|number_inpatient|diag_1|diag_2|diag_3|number_diagn

In [13]:
def columns_with_null_values(df):
    """
    Xem những cột nào có giá trị null trong DataFrame.

    Parameters:
    - df: PySpark DataFrame

    Returns:
    - List: Tên của các cột có giá trị null
    """
    columns_with_null = []

    for column in df.columns:
        null_count = df.filter(col(column).isNull()).count()
        if null_count > 0:
            columns_with_null.append(column)

    return columns_with_null
columns_with_null = columns_with_null_values(df)

# Hiển thị kết quả
if columns_with_null:
    print("Các cột có giá trị null:")
    for column in columns_with_null:
        print(column)
else:
    print("Không có cột nào có giá trị null.")


Các cột có giá trị null:
race
weight
payer_code
medical_specialty
diag_1
diag_2
diag_3


In [14]:
"""

import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

# Assuming 'df' is your PySpark DataFrame
imputer = MissForest()
null_columns = ['race', 'weight', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3']

import numpy as np

for column in null_columns:
    column_values = df.select(column).rdd.flatMap(lambda x: x).collect()

    # Convert the result to a NumPy array
    numpy_array = np.array(column_values)

    # Impute missing values
    numpy_array_imputed = imputer.fit_transform(numpy_array.reshape(-1, 1)).flatten()

    # Update the original DataFrame with the imputed values
    df = df.withColumn(column, F.array([F.lit(value) for value in numpy_array_imputed]))

# Display the updated DataFrame
df.show()
"""


"\n\nimport sklearn.neighbors._base\nimport sys\nsys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base\nfrom missingpy import MissForest\n\n# Assuming 'df' is your PySpark DataFrame\nimputer = MissForest()\nnull_columns = ['race', 'weight', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3']\n\nimport numpy as np\n\nfor column in null_columns:\n    column_values = df.select(column).rdd.flatMap(lambda x: x).collect()\n\n    # Convert the result to a NumPy array\n    numpy_array = np.array(column_values)\n\n    # Impute missing values\n    numpy_array_imputed = imputer.fit_transform(numpy_array.reshape(-1, 1)).flatten()\n\n    # Update the original DataFrame with the imputed values\n    df = df.withColumn(column, F.array([F.lit(value) for value in numpy_array_imputed]))\n\n# Display the updated DataFrame\ndf.show()\n"

### Based on [2014 beata strack et al], there are 22 important columns that afffect readmission rates

In [ ]:
imp_col = ['race','gender','age','admission_type_id','discharge_disposition_id',
           'admission_source_id','time_in_hospital','num_lab_procedures','num_procedures',
           'num_medications','number_outpatient','number_emergency','number_inpatient',
           'diag_1','diag_2','diag_3','number_diagnoses','max_glu_serum','A1Cresult','insulin','change', 'diabetesMed','readmitted']
unique_counts = [df.select(column_name).distinct().count() for column_name in imp_col]

unique = spark.createDataFrame(list(zip(imp_col, unique_counts)), ["Column", "Unique_Values_Count"])
unique.show(22)

In [ ]:

for column in imp_col:
    column_counts = df.groupBy(column).count()


    print(f"Counts for Column '{column}':")
    column_counts.show(30)

In [17]:
train_df = df.select(imp_col)

In [18]:
for column, data_type in train_df.dtypes:
    print(f"{column}: {data_type}")

race: string
gender: string
age: string
admission_type_id: int
discharge_disposition_id: int
admission_source_id: int
time_in_hospital: int
num_lab_procedures: int
num_procedures: int
num_medications: int
number_outpatient: int
number_emergency: int
number_inpatient: int
diag_1: string
diag_2: string
diag_3: string
number_diagnoses: int
max_glu_serum: string
A1Cresult: string
insulin: string
change: string
diabetesMed: string
readmitted: string


In [19]:
from pyspark.ml.feature import StringIndexer

string_cols = ['race', 'gender', 'age',
               'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'insulin', 'change', 'diabetesMed', 'readmitted']

df_transformed = train_df  # Initialize with the original DataFrame

for column in string_cols:
    string_indexer = StringIndexer(inputCol=column, outputCol=f"{column}_new", handleInvalid="keep")
    model = string_indexer.fit(df_transformed)
    df_transformed = model.transform(df_transformed)

columns_to_drop = ['race', 'gender', 'age',
                   'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'insulin', 'change', 'diabetesMed', 'readmitted']

df_transformed = df_transformed.drop(*columns_to_drop)

# Hiển thị DataFrame sau khi chuyển đổi

#df_transformed = df_transformed.withColumn("race_new", when(col("race_new") == "5.0", None).otherwise(col("race_new")))
df_transformed.show()

+-----------------+------------------------+-------------------+----------------+------------------+--------------+---------------+-----------------+----------------+----------------+----------------+--------+----------+-------+----------+----------+----------+-----------------+-------------+-----------+----------+---------------+--------------+
|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|num_lab_procedures|num_procedures|num_medications|number_outpatient|number_emergency|number_inpatient|number_diagnoses|race_new|gender_new|age_new|diag_1_new|diag_2_new|diag_3_new|max_glu_serum_new|A1Cresult_new|insulin_new|change_new|diabetesMed_new|readmitted_new|
+-----------------+------------------------+-------------------+----------------+------------------+--------------+---------------+-----------------+----------------+----------------+----------------+--------+----------+-------+----------+----------+----------+-----------------+-------------+-----------

In [ ]:
from pyspark.ml.feature import StringIndexer

string_cols = ['race', 'gender', 'age',
               'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'insulin', 'change', 'diabetesMed', 'readmitted']

# Initialize the DataFrame to the original DataFrame
final_df = train_df

# Loop through each column and add the new column to the DataFrame
for column in string_cols:
    string_indexer = StringIndexer(inputCol=column, outputCol=f"{column}_new", handleInvalid="keep")
    model = string_indexer.fit(final_df)
    final_df = model.transform(final_df)
final_df = final_df.drop(*string_cols)
# Perform groupBy operation on the final DataFrame
for column in final_df.columns:
    column_counts = final_df.groupBy(column).count()
    print(f"Counts for Column '{column}':")
    column_counts.show(30)


In [21]:
from pyspark.sql.functions import when, col


final_df = final_df.withColumn('race_new', when((col('race_new') == 5.0), None).otherwise(col('race_new')))
final_df = final_df.withColumn('diag_1_new', when((col('diag_1_new') == 695.0), None).otherwise(col('diag_1_new')))
final_df = final_df.withColumn('diag_2_new', when((col('diag_2_new') == 718.0), None).otherwise(col('diag_2_new')))
final_df = final_df.withColumn('diag_3_new', when((col('diag_3_new') == 746.0), None).otherwise(col('diag_3_new')))

for column in final_df.columns:
  null_count = final_df.filter(col(column).isNull()).count()
  if null_count > 0:
    print(f"{column} has {null_count} missing data")

race_new has 1878 missing data
diag_1_new has 17 missing data
diag_2_new has 290 missing data
diag_3_new has 1146 missing data


In [22]:
!pip install missingpy

In [23]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [44]:
from pyspark.sql.functions import col

columns_to_convert = ['race_new', 'diag_1_new', 'diag_2_new', 'diag_3_new']
data_list = final_df.select(*[col(c) for c in columns_to_convert]).collect()

data_list_python = [list(row) for row in data_list]
# Chuyển đổi danh sách theo chiều ngang
data_list_transposed = list(map(list, zip(*data_list_python)))

# In kết quả
for row in data_list_transposed:
    print(row)


[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, None, None, None, 1.0, 0.0, 1.0, 3.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, None, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, None, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 3.0, 0.0, 0.0, None, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 3.0, None, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, None, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 3.0, 0.0, None, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, None, 0.0, 0.0, 1.0, 0.0, 1.0, None, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 1.0, 1.0, None, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 1.

In [33]:
np_array_transposed = np.array(data_list_transposed)

In [ ]:

data_list = final_df.collect()

data_list_python = [list(row) for row in data_list]
data_list_python

In [47]:
print(X)
print(data_list_python)

[[1, 2, None], [3, 4, 3], [None, 6, 5], [8, 8, 7]]
[[0.0, 159.0, None, None], [0.0, 12.0, 15.0, 180.0], [1.0, 62.0, 0.0, 189.0], [0.0, 45.0, 273.0, 8.0], [0.0, 55.0, 136.0, 0.0], [0.0, 0.0, 8.0, 0.0], [0.0, 0.0, 8.0, 12.0], [0.0, 1.0, 65.0, 0.0], [0.0, 129.0, 3.0, 96.0], [0.0, 7.0, 52.0, 30.0], [1.0, 32.0, 9.0, 64.0], [1.0, 116.0, 161.0, 35.0], [0.0, 1.0, 8.0, 4.0], [1.0, 19.0, 28.0, 308.0], [1.0, 272.0, 69.0, 64.0], [1.0, 3.0, 8.0, 5.0], [0.0, 9.0, 247.0, 0.0], [1.0, 46.0, 21.0, 95.0], [None, 427.0, 3.0, 143.0], [None, 3.0, 3.0, 3.0], [None, 91.0, 163.0, 4.0], [1.0, 3.0, 4.0, 362.0], [0.0, 17.0, 105.0, 72.0], [1.0, 112.0, 6.0, 4.0], [3.0, 2.0, 4.0, 0.0], [0.0, 5.0, 2.0, 5.0], [0.0, 1.0, 30.0, 5.0], [1.0, 16.0, 6.0, 132.0], [1.0, 348.0, 15.0, 2.0], [0.0, 1.0, 13.0, 5.0], [0.0, 101.0, 124.0, 0.0], [0.0, 31.0, 44.0, 5.0], [0.0, 11.0, 9.0, 101.0], [0.0, 0.0, 8.0, 21.0], [0.0, 76.0, 50.0, 10.0], [0.0, 17.0, 1.0, 111.0], [0.0, 135.0, 39.0, 4.0], [0.0, 141.0, 15.0, 47.0], [0.0, 52.0, 5.0, 0.

In [ ]:
from pyspark.sql import functions as F

from missingpy import MissForest
imputer = MissForest()

imputed = imputer.fit_transform(data_list_python)

In [ ]:
from pyspark.sql import functions as F

from missingpy import MissForest
imputer = MissForest(criterion='squared_error',max_features=1.0)

imputed = imputer.fit_transform(data_list_python)

In [55]:
imputed

array([[ 6., 25.,  1., ...,  0.,  1.,  0.],
       [ 1.,  1.,  7., ...,  1.,  0.,  1.],
       [ 1.,  1.,  7., ...,  0.,  0.,  0.],
       ...,
       [ 1.,  1.,  7., ...,  1.,  0.,  0.],
       [ 2.,  3.,  7., ...,  1.,  0.,  0.],
       [ 1.,  1.,  7., ...,  0.,  1.,  0.]])

In [67]:
from pyspark.sql.types import StructType, StructField, FloatType

schema = StructType([
    StructField("admission_type_id", FloatType(), True),
    StructField("discharge_disposition_id", FloatType(), True),
    StructField("admission_source_id", FloatType(), True),
    StructField("time_in_hospital", FloatType(), True),
    StructField("num_lab_procedures", FloatType(), True),
    StructField("num_procedures", FloatType(), True),
    StructField("num_medications", FloatType(), True),
    StructField("number_outpatient", FloatType(), True),
    StructField("number_emergency", FloatType(), True),
    StructField("number_inpatient", FloatType(), True),
    StructField("number_diagnoses", FloatType(), True),
    StructField("race_new", FloatType(), True),
    StructField("gender_new", FloatType(), True),
    StructField("age_new", FloatType(), True),
    StructField("diag_1_new", FloatType(), True),
    StructField("diag_2_new", FloatType(), True),
    StructField("diag_3_new", FloatType(), True),
    StructField("max_glu_serum_new", FloatType(), True),
    StructField("A1Cresult_new", FloatType(), True),
    StructField("insulin_new", FloatType(), True),
    StructField("change_new", FloatType(), True),
    StructField("diabetesMed_new", FloatType(), True),
    StructField("readmitted_new", FloatType(), True)
])

# Tạo DataFrame từ mảng numpy và schema
train_data = spark.createDataFrame(imputed.tolist(), schema=schema)

# In DataFrame
train_data.show()

+-----------------+------------------------+-------------------+----------------+------------------+--------------+---------------+-----------------+----------------+----------------+----------------+--------+----------+-------+----------+----------+----------+-----------------+-------------+-----------+----------+---------------+--------------+
|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|num_lab_procedures|num_procedures|num_medications|number_outpatient|number_emergency|number_inpatient|number_diagnoses|race_new|gender_new|age_new|diag_1_new|diag_2_new|diag_3_new|max_glu_serum_new|A1Cresult_new|insulin_new|change_new|diabetesMed_new|readmitted_new|
+-----------------+------------------------+-------------------+----------------+------------------+--------------+---------------+-----------------+----------------+----------------+----------------+--------+----------+-------+----------+----------+----------+-----------------+-------------+-----------

In [ ]:
for column in train_data.columns:
    column_counts = train_data.groupBy(column).count()
    print(f"Counts for Column '{column}':")
    column_counts.show(30)

In [61]:

for column in train_data.columns:
  null_count = train_data.filter(col(column).isNull()).count()
  count = 0
  if null_count > 0:
    print(f"{column} has {null_count} missing data")
    count+=1
if count==0:
  print('All data have been filled')

All data have been filled


In [69]:
for column, data_type in train_data.dtypes:
    print(f"{column}: {data_type}")

admission_type_id: float
discharge_disposition_id: float
admission_source_id: float
time_in_hospital: float
num_lab_procedures: float
num_procedures: float
num_medications: float
number_outpatient: float
number_emergency: float
number_inpatient: float
number_diagnoses: float
race_new: float
gender_new: float
age_new: float
diag_1_new: float
diag_2_new: float
diag_3_new: float
max_glu_serum_new: float
A1Cresult_new: float
insulin_new: float
change_new: float
diabetesMed_new: float
readmitted_new: float


In [70]:
from pyspark.sql.functions import expr

df_processed = train_data.withColumn('race_new', expr('CAST(ROUND(race_new) AS INT)'))

# Hiển thị DataFrame sau khi xử lý
df_processed.show()

+-----------------+------------------------+-------------------+----------------+------------------+--------------+---------------+-----------------+----------------+----------------+----------------+--------+----------+-------+----------+----------+----------+-----------------+-------------+-----------+----------+---------------+--------------+
|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|num_lab_procedures|num_procedures|num_medications|number_outpatient|number_emergency|number_inpatient|number_diagnoses|race_new|gender_new|age_new|diag_1_new|diag_2_new|diag_3_new|max_glu_serum_new|A1Cresult_new|insulin_new|change_new|diabetesMed_new|readmitted_new|
+-----------------+------------------------+-------------------+----------------+------------------+--------------+---------------+-----------------+----------------+----------------+----------------+--------+----------+-------+----------+----------+----------+-----------------+-------------+-----------

In [72]:
train_data = df_processed
column_counts = train_data.groupBy('race_new').count()
print(f"Counts for Column 'race_new':")
column_counts.show(30)

Counts for Column 'race_new':
+--------+-----+
|race_new|count|
+--------+-----+
|       1|13337|
|       3| 1167|
|       4|  512|
|       2| 1508|
|       0|54994|
+--------+-----+



In [84]:

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline


# Điều chỉnh tên cột 'readmitted_new' thành 'label' để sử dụng trong mô hình
train_data = train_data.withColumnRenamed("readmitted_new", "label")

# Chọn các cột đặc trưng
feature_columns = ["admission_type_id", "discharge_disposition_id", "admission_source_id",
                    "time_in_hospital", "num_lab_procedures", "num_procedures", "num_medications",
                    "number_outpatient", "number_emergency", "number_inpatient", "number_diagnoses",
                    "race_new", "gender_new", "age_new", "diag_1_new", "diag_2_new", "diag_3_new",
                    "max_glu_serum_new", "A1Cresult_new", "insulin_new", "change_new", "diabetesMed_new"]

# Tạo VectorAssembler để kết hợp các cột đặc trưng thành một cột Vector
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Tạo mô hình Logistic Regression
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Tạo pipeline
pipeline = Pipeline(stages=[assembler, lr])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
(training_data, test_data) = train_data.randomSplit([0.8, 0.2], seed=42)

# Huấn luyện mô hình
model = pipeline.fit(training_data)

# Dự đoán trên tập kiểm tra
predictions = model.transform(test_data)

# Hiển thị kết quả
column_counts = predictions.groupBy("prediction").count()

column_counts.show(30)
column_count = predictions.groupBy("label").count()

column_count.show(30)

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|13977|
|       1.0|  219|
|       2.0|    6|
+----------+-----+

+-----+-----+
|label|count|
+-----+-----+
|  2.0|  675|
|  1.0| 2765|
|  0.0|10762|
+-----+-----+



In [87]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Chuyển đổi dữ liệu dự đoán thành Pandas DataFrame để sử dụng sklearn.metrics
predictions_pd = predictions.select("label", "prediction").toPandas()

# Lấy nhãn thực tế và dự đoán
y_true = predictions_pd["label"]
y_pred = predictions_pd["prediction"]

# Tính confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# In confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Tính các độ đo
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# In các độ đo
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Confusion Matrix:
[[10675    84     3]
 [ 2661   103     1]
 [  641    32     2]]
Accuracy: 0.7590480214054358
Precision: 0.6861683073566933
Recall: 0.7590480214054358
F1 Score: 0.6676917219586354


In [107]:
test_path = "/content/drive/MyDrive/Big Data /Kaggle/data/test.csv"

test_df = spark.read.csv(test_path, header=True, inferSchema=True)

question_mark_counts = count_question_marks_per_column(test_df)
for column, count in question_mark_counts.items():
    if count >0:
      print(f"Số lượng giá trị '?' trong cột '{column}': {count}")
test_df = test_df.withColumn("weight", when(col("weight") == "?", None).otherwise(col("weight")))
test_df = test_df.withColumn("race", when(col("race") == "?", None).otherwise(col("race")))
test_df = test_df.withColumn("payer_code", when(col("payer_code") == "?", None).otherwise(col("payer_code")))
test_df = test_df.withColumn("medical_specialty", when(col("medical_specialty") == "?", None).otherwise(col("medical_specialty")))
test_df = test_df.withColumn("diag_1", when(col("diag_1") == "?", None).otherwise(col("diag_1")))
test_df = test_df.withColumn("diag_2", when(col("diag_2") == "?", None).otherwise(col("diag_2")))
test_df = test_df.withColumn("diag_3", when(col("diag_3") == "?", None).otherwise(col("diag_3")))

imp_col = ['race','gender','age','admission_type_id','discharge_disposition_id',
           'admission_source_id','time_in_hospital','num_lab_procedures','num_procedures',
           'num_medications','number_outpatient','number_emergency','number_inpatient',
           'diag_1','diag_2','diag_3','number_diagnoses','max_glu_serum','A1Cresult','insulin','change', 'diabetesMed']

test_df = test_df.select(imp_col)
from pyspark.ml.feature import StringIndexer

string_cols = ['race', 'gender', 'age',
               'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'insulin', 'change', 'diabetesMed']

df_transformed = test_df  # Initialize with the original DataFrame

for column in string_cols:
    string_indexer = StringIndexer(inputCol=column, outputCol=f"{column}_new", handleInvalid="keep")
    model = string_indexer.fit(df_transformed)
    df_transformed = model.transform(df_transformed)

columns_to_drop = ['race', 'gender', 'age',
                   'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'insulin', 'change', 'diabetesMed']

df_transformed = df_transformed.drop(*columns_to_drop)

from pyspark.ml.feature import StringIndexer

string_cols = ['race', 'gender', 'age',
               'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'insulin', 'change', 'diabetesMed']

# Initialize the DataFrame to the original DataFrame
final_df = test_df

# Loop through each column and add the new column to the DataFrame
for column in string_cols:
    string_indexer = StringIndexer(inputCol=column, outputCol=f"{column}_new", handleInvalid="keep")
    model = string_indexer.fit(final_df)
    final_df = model.transform(final_df)
final_df = final_df.drop(*string_cols)
# Perform groupBy operation on the final DataFrame
for column in final_df.columns:
    column_counts = final_df.groupBy(column).count()
    print(f"Counts for Column '{column}':")
    column_counts.show(30)

from pyspark.sql.functions import when, col

final_df = final_df.withColumn('race_new', when((col('race_new') == 5.0), None).otherwise(col('race_new')))
final_df = final_df.withColumn('diag_1_new', when((col('diag_1_new') == 559.0), None).otherwise(col('diag_1_new')))
final_df = final_df.withColumn('diag_2_new', when((col('diag_2_new') == 547.0), None).otherwise(col('diag_2_new')))
final_df = final_df.withColumn('diag_3_new', when((col('diag_3_new') == 605.0), None).otherwise(col('diag_3_new')))




Số lượng giá trị '?' trong cột 'race': 395
Số lượng giá trị '?' trong cột 'weight': 29898
Số lượng giá trị '?' trong cột 'payer_code': 10171
Số lượng giá trị '?' trong cột 'medical_specialty': 15424
Số lượng giá trị '?' trong cột 'diag_1': 4
Số lượng giá trị '?' trong cột 'diag_2': 68
Số lượng giá trị '?' trong cột 'diag_3': 277
Counts for Column 'admission_type_id':
+-----------------+-----+
|admission_type_id|count|
+-----------------+-----+
|                1|16799|
|                6| 1009|
|                3| 5046|
|                5| 1926|
|                4|    1|
|                8|   17|
|                7|    2|
|                2| 5448|
+-----------------+-----+

Counts for Column 'discharge_disposition_id':
+------------------------+-----+
|discharge_disposition_id|count|
+------------------------+-----+
|                      28|   61|
|                      27|    1|
|                      12|    1|
|                      22|  714|
|                       1|16820|
|      

In [109]:
data_list = final_df.collect()

data_list_python = [list(row) for row in data_list]
from pyspark.sql import functions as F

from missingpy import MissForest
imputer = MissForest(criterion='squared_error',max_features=1.0)

imputed_test = imputer.fit_transform(data_list_python)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4


In [110]:
from pyspark.sql.types import StructType, StructField, FloatType

schema = StructType([
    StructField("admission_type_id", FloatType(), True),
    StructField("discharge_disposition_id", FloatType(), True),
    StructField("admission_source_id", FloatType(), True),
    StructField("time_in_hospital", FloatType(), True),
    StructField("num_lab_procedures", FloatType(), True),
    StructField("num_procedures", FloatType(), True),
    StructField("num_medications", FloatType(), True),
    StructField("number_outpatient", FloatType(), True),
    StructField("number_emergency", FloatType(), True),
    StructField("number_inpatient", FloatType(), True),
    StructField("number_diagnoses", FloatType(), True),
    StructField("race_new", FloatType(), True),
    StructField("gender_new", FloatType(), True),
    StructField("age_new", FloatType(), True),
    StructField("diag_1_new", FloatType(), True),
    StructField("diag_2_new", FloatType(), True),
    StructField("diag_3_new", FloatType(), True),
    StructField("max_glu_serum_new", FloatType(), True),
    StructField("A1Cresult_new", FloatType(), True),
    StructField("insulin_new", FloatType(), True),
    StructField("change_new", FloatType(), True),
    StructField("diabetesMed_new", FloatType(), True)
])

# Tạo DataFrame từ mảng numpy và schema
test = spark.createDataFrame(imputed_test.tolist(), schema=schema)

from pyspark.sql.functions import expr

df_processed = test.withColumn('race_new', expr('CAST(ROUND(race_new) AS INT)'))

# Hiển thị DataFrame sau khi xử lý
df_processed.show()

+-----------------+------------------------+-------------------+----------------+------------------+--------------+---------------+-----------------+----------------+----------------+----------------+--------+----------+-------+----------+----------+----------+-----------------+-------------+-----------+----------+---------------+
|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|num_lab_procedures|num_procedures|num_medications|number_outpatient|number_emergency|number_inpatient|number_diagnoses|race_new|gender_new|age_new|diag_1_new|diag_2_new|diag_3_new|max_glu_serum_new|A1Cresult_new|insulin_new|change_new|diabetesMed_new|
+-----------------+------------------------+-------------------+----------------+------------------+--------------+---------------+-----------------+----------------+----------------+----------------+--------+----------+-------+----------+----------+----------+-----------------+-------------+-----------+----------+---------------+
|

In [113]:
# Tạo VectorAssembler để kết hợp các cột đặc trưng thành một cột Vector
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Tạo mô hình Logistic Regression
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Tạo pipeline
pipeline = Pipeline(stages=[assembler, lr])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
(training_data, test_data) = train_data.randomSplit([0.8, 0.2], seed=42)

# Huấn luyện mô hình
model = pipeline.fit(training_data)

# Dự đoán trên tập kiểm tra
prediction = model.transform(df_processed)
prediction = predictions.select('prediction')
prediction.show(30)

+----------+
|prediction|
+----------+
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       1.0|
+----------+
only showing top 30 rows



In [119]:
prediction = prediction.withColumn(
    'prediction',
    when(prediction['prediction'] == 0, 'No')
    .when(prediction['prediction'] == 1, 'GT30')
    .when(prediction['prediction'] == 2, 'LT30')
    .otherwise(prediction['prediction'])
)

# Hiển thị kết quả
prediction.show()

+----------+
|prediction|
+----------+
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
|        No|
+----------+
only showing top 20 rows



In [124]:
from pyspark.sql.functions import monotonically_increasing_id

df_with_index = prediction.withColumn("index", monotonically_increasing_id())

submit_df = df_with_index.select("index", "prediction")
submit_df.show()

+-----+----------+
|index|prediction|
+-----+----------+
|    0|        No|
|    1|        No|
|    2|        No|
|    3|        No|
|    4|        No|
|    5|        No|
|    6|        No|
|    7|        No|
|    8|        No|
|    9|        No|
|   10|        No|
|   11|        No|
|   12|        No|
|   13|        No|
|   14|        No|
|   15|        No|
|   16|        No|
|   17|        No|
|   18|        No|
|   19|        No|
+-----+----------+
only showing top 20 rows



In [125]:
for column in submit_df.columns:
    column_counts = submit_df.groupBy(column).count()
    print(f"Counts for Column '{column}':")
    column_counts.show(30)

Counts for Column 'index':
+-----+-----+
|index|count|
+-----+-----+
|   26|    1|
|   29|    1|
|  474|    1|
|  964|    1|
| 1677|    1|
| 1697|    1|
| 1806|    1|
| 1950|    1|
| 2040|    1|
| 2214|    1|
| 2250|    1|
| 2453|    1|
| 2509|    1|
| 2529|    1|
| 2927|    1|
| 3091|    1|
| 3506|    1|
| 3764|    1|
| 4590|    1|
| 4823|    1|
| 4894|    1|
| 5385|    1|
| 5409|    1|
| 5556|    1|
| 6721|    1|
|   65|    1|
|  191|    1|
|  418|    1|
|  541|    1|
|  558|    1|
+-----+-----+
only showing top 30 rows

Counts for Column 'prediction':
+----------+-----+
|prediction|count|
+----------+-----+
|      LT30|    6|
|        No|13977|
|      GT30|  219|
+----------+-----+



In [128]:
# Ghi DataFrame vào tệp CSV

submit_df.write.option("header",True).csv("/content/drive/MyDrive/Big Data /Kaggle/data/submit")
